In [ ]:
#importing essential packages
import numpy as np
import pandas as pd

In [ ]:
#reading dataset
df=pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')


# **1)EDA**

# A.Understanding the data

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

# **B.Visualizing**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Finding correlation
correl=df.corr()
plt.subplots(figsize=(16,10))
fig1=sns.heatmap(correl, annot=True)
bottom, top = fig1.get_ylim()
fig1.set_ylim(bottom + 0.5, top - 0.5)

In [ ]:
print("UNIVARIATE ANALYSIS-HISTOGRAMS:")
for i in [0,1,2,9,10]:
    plt.subplots(figsize=(10,5))
    sns.distplot(df.iloc[:,i],color='purple',bins=15)
    plt.show()
    

In [ ]:
print("BOXPLOTS:")
for i in range(0,12):
    plt.subplots(figsize=(8,4))
    sns.boxplot(x=df.iloc[:,11],y = df.iloc[:,i],palette="cool")
    plt.show()

In [ ]:
print("BIVARIATE ANALYSIS: Scatter plots-\n")
l1=[1,2,9,10]
for i in l1:
    for j in l1:
        if(j!=i):
            sns.scatterplot(x=df.iloc[:,i],y=df.iloc[:,j],hue=df["quality"])
            plt.show()
    l1.remove(i)

# **C.Feature selection and data scaling**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 
#Feature selection
y=df.iloc[:,11]
X=df.iloc[:,[1,2,9,10]] #Using only top 4 columns with highest correlation to quality


In [ ]:
#Scaling the data
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X=ss.fit_transform(X)

# **2.Model creation**


Note: Usually the random state is not a parameter to be tuned, and will produced 'optimistically biased' results. I have done it here anyway to give a best case scenario.
You can find out more from random state manipulation from these links:
[https://towardsdatascience.com/manipulating-machine-learning-results-with-random-state-2a6f49b31081](http://)
and 
[https://stats.stackexchange.com/questions/263999/is-random-state-a-parameter-to-tune](http://)


In [ ]:

#Finding best suitable random state between state 1-43

"""This step fits multiple KNN models with number of neighbours ranging from 1-35 and random states 1-43
to give the best case accuracy. Random state manipulation is NOT A GOOD PRACTICE but FINDING OPTIMAL NUMBER OF NEIGHBOURS
for the particular problem is important. 

PLEASE NOTE THIS STEP TAKES AROUND 1 MINUTE TO RUN"""

state_macc=list(np.empty(43))
for i in range(1,43):
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=i,stratify=y)
    
    neighbors = np.arange(1,35)
    train_accuracy = np.empty(len(neighbors))
    test_accuracy = np.empty(len(neighbors))
    for j,k in enumerate(neighbors):
        knn2 = KNeighborsClassifier(n_neighbors=k,weights='distance',metric='manhattan')
        knn2.fit(X_train,y_train)
        train_accuracy[j] = knn2.score(X_train, y_train)
        test_accuracy[j] = knn2.score(X_test, y_test)
    macc=np.amax(test_accuracy)
    x=np.where(test_accuracy==np.amax(test_accuracy))
    x=x[0].tolist()
    x = [i+1 for i in x]
    data=[x,float(macc)]
    state_macc[i-1]=data


In [ ]:
temp=0.0
pos=0
for i in range(42):
    state_macc[i][-1] = float(state_macc[i][-1])
    if (state_macc[i][-1] > temp):
        temp = state_macc[i][-1]
        pos=i+1
ideal_k=state_macc[pos-1][0]
print("Ideal random state= {} yielding max accuracy {} with number of neighbours: ".format(pos,temp),ideal_k)    

# Final Model:- Accuracy of about 75%

In [ ]:
"""THEREFORE Final model is prepared using ideal Random State and Best K number of neighbours """

from sklearn.neighbors import KNeighborsClassifier 
y=df.iloc[:,11]
X=df.iloc[:,[1,2,9,10]] #Using only top 4 columns with highest correlation to quality

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X=ss.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=pos,stratify=y)

knn = KNeighborsClassifier(n_neighbors=ideal_k[0],weights='distance',metric='manhattan')
knn.fit(X_train,y_train)

facc=knn.score(X_test,y_test)
print("\n\nAccuracy of final model is: ", facc*100, "%\n\n")

In [ ]:
print("Prediction table:")
a=knn.predict(X_test)
b=y_test
z=list(enumerate(a))
z=[x[0] for x in z]
table={'Index':z,'Predicted value':a,'Real value':b}
nf=pd.DataFrame(table)
nf.style.hide_index()

# Conclusion
**As we can see the KNN model gives us an accuracy of about 75% on the test data, which means that makes correct predictions for roughly 3 in 4 datapoints. Another point to be noted from the prediction table is that most of the wrong predictions aren't off by much (i.e. a 6 might be predicted as a 5, not as a 3) which indicates that the predictions are still pretty close to the true data. We can note that though the KNN model is 'decent'in performance, there are other algorithms that can be used and improvements that can be made.**